In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np

## Frequency Domain and Frequency Sharing

Lets repeat the example from the previous lecture where we set some parameters as follows:

In [ ]:
f_0=1e9       # Hz- carrier frequency
T=10*1e-9     # s - duration of one bit during transmission in seconds

Ts=1/(4*f_0)  # s - sampling period in seconds - high sampling rate for visualization purposes

# generate some random data
bits = np.random.binomial(n=1, p=0.5, size=512)

print('Number of bits : {} , the first 4 bits: {}'.format(len(bits),bits[0:4]))

The simplest modulation we used is on-off keying where the sinusoidal signal is multiplied with the bit values. It is a form of amplitude modulation.

In [ ]:
# generate the modulated signal
s_AM = lambda t: bits[np.floor_divide(t,T).astype('int')]*np.cos(2*np.pi*f_0*t)

# show the first 4 bits
t=np.arange(0,4*T,Ts)
plt.plot(t*1e9,s_AM(t))
plt.xlabel('Time[ns]')
plt.title('Amplitude modualtion - the first 4 bits: {}'.format(bits[0:4]))
plt.show()

print('Communcation speed: {} Mbits/s'.format(1e-6/T))

Lets look now how this signal looks in the frequency domain:

In [ ]:
def pplotF(s,Ts,T='',f_0=''):
    
    Ns = len(s)
    S  = np.fft.fft(s)
    P  = (2.0/Ns * np.abs(S[0:Ns//2]))**2

    plt.plot(np.linspace(0.0, 1.0/(2.0*Ts), Ns//2)/1e6, P)
    plt.grid()
    plt.xlabel('MHz')
    plt.title('Signal power in the frequency domain')
    if T:
        plt.gca().add_patch(patches.Rectangle([(f_0-0.5/T)/1e6,0],(1/T)/1e6,max(P),color='r',alpha=0.5))
    plt.show()

pplotF(s_AM(np.arange(0,len(bits)*T,Ts)),Ts,T,f_0)

## Amplitude and Phase Modulation

For the typical sinusoidal signals, the infomation can be embedded in the amplitude and the phase. 

$$ A(t)\sin(phase(t)) $$ 

Lets first try the previous example but now with phase modulation where we shift the phase by $\pi$ for every bit. This is also known as binary phase shift keying modulation (BPSK).

In [ ]:
# phase modulation
s_PM = lambda t: np.cos(2*np.pi*f_0*t + np.pi*bits[np.floor_divide(t,T).astype('int')])

# show the first 4 bits
t=np.arange(0,4*T,Ts)
plt.plot(t*1e9,s_PM(t))
plt.xlabel('Time[ns]')
plt.title('Phase modualtion - the first 4 bits: {}'.format(bits[0:4]))
plt.show()

Lets look now how this signal looks in the frequency domain:

In [ ]:
pplotF(s_PM(np.arange(0,len(bits)*T,Ts)),Ts,T,f_0)

The comunication speed depends on the duration of a bit $T$ in the modulated signal. The frequency bandwidth needed for certain comunication speed will be proportional to the communication speed. Higher speed leads to short bits and that will require wider set of frequencies to be present in the signal. To illustrate this we construct now a 4 times faster phase modulated signal and show the signal spectrum. 

In [ ]:
T_faster=T/4
s_PM_faster = lambda t: np.cos(2*np.pi*f_0*t + np.pi*bits[np.floor_divide(t,T_faster).astype('int')])

pplotF(s_PM_faster(np.arange(0,len(bits)*T_faster,Ts)),Ts,T_faster,f_0)

# Quadrature Amplitude Modulation

We can think of more complex modulation schemes where we change the phase and the amplitude of the signal. Since phase and amplitude are very different, a practical represetnation is to use the IQ form:

$$ I(t)\cos(2\pi f_0 t) + Q(t)\sin(2\pi f_0 t) $$

The capital letter I represents the amplitude of the in-phase signal, and the capital letter Q represents the amplitude of the quadrature signal. 

Furthermore a pair $(I,Q)$ is typically represented as a single complex number.

Finally, this is also how the typical IQ transmiiter and recever are realized in analog electronics.

The transmitter could be realised by mixing I and Q signals like this:

![alt_text](images/QTX.svg)

At the receiver we can extract the analog I and Q signals in the similar way:

![alt_text](images/QRX.svg)


If we consider now the ambilitudes of the I and Q signals as cooridinates in a 2D space, we can generate many complex modulation schemes. The most typical are dividing the space into equidistant grid, e.g. QPSK, QAM16, QAM64, ... 

Modulation schemes having more than 2 states will introduce the concept of a symbol where a number of bits are grouped into a symbol. Here is an example QAM scheme:

In [ ]:
# QAM 
NQAM=16
bits_per_symbol = np.floor(np.log2(NQAM)).astype(int)

# serial to parallel - group the bits
bit_groups = bits.reshape(-1,bits_per_symbol)
# convert the bits representation to symbol index
symbols = np.array([ int(''.join([str(bit) for bit in bit_group]),2) for bit_group in bit_groups ])

print('Bits: {} , Symbols: {} , Bits per symbol: {}'.format(len(bits),symbols.shape[0],bits_per_symbol))
print('Communcation speed: {} Msymbols/s, {} Mbits/s'.format(1e-6/T,bits_per_symbol*1e-6/T,))

We will transmmit each symbol during the same period $T$. As now each symbol is caring information of a few bits the communication speed measured in transmitted bits is increased.

Let's now create a mapping from symbol index to a complex number representing I and Q signals.

In [ ]:
# construct IQ lookup
r    = np.linspace(-1, 1, np.sqrt(NQAM).astype(int))
I, Q = np.meshgrid(r, r)
IQ   = I.flatten()+1j*Q.flatten()

# convert the symbols to complex number representation
Symbols = IQ[symbols]

# show
plt.plot(IQ.real, IQ.imag, 'o')
for i in range(16):
    plt.text(IQ[i].real, IQ[i].imag, format(i, '04b'))
plt.grid()
plt.xlabel('I')
plt.ylabel('Q')
plt.show()

A schematic of the described operations and a transmitter circuit looks like this:

![alt_text](images/QAMTX.svg)

And example signal can be constructed like this:

In [ ]:
# QAM signal
s_QAM = lambda t: np.real(Symbols[np.floor_divide(t,T).astype('int')] * np.exp(2j*np.pi*f_0*t))

# show the first 4 symbols
print(Symbols[0:4])
t=np.arange(0,4*T,Ts)
plt.plot(t*1e9,s_QAM(t))
plt.xlabel('Time[ns]')
plt.title('QAM modulation - the first 4 symbols')
plt.show()

And in the frequency domain:

In [ ]:
pplotF(s_QAM(np.arange(0,len(symbols)*T,Ts)),Ts,T,f_0)

## Orthogonal Frequency Division Multiplexing

The main idea is to divide the given bandwith into set of smaller bands and then transmmit at each of sub-bands (sub-carriers) at lower speed. This gives lower speed per subcarrier but multiple symbols are transmitted simultaneously.

![alt_text](images/subcarriers.svg)


This might look complex to generate but there exist a simple trick to generate such signals using inverse FFT. A set of $N_subcarrier$ complex simbols is considered as a frequency domain representation. The set is then transformed to time representation using inverse FFT achieving exactly the desired effect. 

![alt_text](images/OFDMTX.svg)


In [ ]:
Nsubcarriers=4

# serial to parallel - group the symbols
Symbol_groups = Symbols.reshape(-1,Nsubcarriers)

# frequency to time
SymbolOFDM = np.fft.ifft(Symbol_groups,axis=1).flatten()

# OFDM signal
s_OFDM = lambda t: np.real(SymbolsOFDM[np.floor_divide(t,T).astype('int')] * np.exp(2j*np.pi*f_0*t))

# show the first 4 symbols
print(SymbolsOFDM[0:4])
t=np.arange(0,4*T,Ts)
plt.plot(t*1e9,s_OFDM(t))
plt.xlabel('Time[ns]')
plt.title('OFDM modulation - the first 4 symbols combined and sent together')
plt.show()

In [ ]:
pplotF(s_OFDM(np.arange(0,len(symbols)*T,Ts)),Ts,T,f_0)